In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-credit-scoring/bank.csv
/kaggle/input/bank-credit-scoring/scoring.jpg


First, let's look at the data and preprocess it

In [2]:
df = pd.read_csv("/kaggle/input/bank-credit-scoring/bank.csv", sep=";")
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


Let's see what categorial and numerical features we have and how much their values vary

In [3]:
cat_features = df.columns[df.dtypes == 'object']
num_features = df.columns[df.dtypes != 'object']
cat_features

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')

In [4]:
data = [(feature, "categorial", df.loc[:, feature].nunique()) for feature in cat_features] + \
       [(feature, "numerical", f"({df.loc[:, feature].min()}; {df.loc[:, feature].max()})") for feature in num_features]

values_df = pd.DataFrame(
    data=data,
    columns=["feature", "type", "uniqie values / range"]
).set_index("feature").reindex(df.columns)

values_df

,type,uniqie values / range
age,numerical,(19; 87)
job,categorial,12
marital,categorial,3
education,categorial,4
default,categorial,2
balance,numerical,(-3313; 71188)
housing,categorial,2
loan,categorial,2
contact,categorial,3
day,numerical,(1; 31)


We see that there is not much variance in categorial data so we can use one hot encoding. The numerical values look normal, but we had better normilize the data.